In [1]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier


import os
import time
import numpy as np
import cv2
from skimage import feature, io, color
from sklearn.metrics import classification_report
import joblib
import json
import itertools
from tqdm import tqdm

from utils import *

In [2]:
# test_dir = "../data/test"
# test_data, test_labels = load_raw_data(test_dir)

train_dir = "../data/train"
train_data, train_labels = load_raw_data(train_dir)
train_labels = np.array(train_labels)

bins = [8, 16, 32, 64, 128, 256]
sizes = [32, 64, 128, 256, 512]  

# bins_h = [8, 16, 32, 64, 128, 180]
# bins_s = [8, 16, 32, 64, 128, 256]
# bins_v = [8, 16, 32, 64, 128, 256]

# bin_combinations = list(itertools.product(bins_h, bins_s, bins_v))

# *SVM*

In [3]:
results = []
start = time.time()

for bin in tqdm(bins, desc="Processing"):
    for img_size in sizes:
        svm = SVC()

        train_data_ = [preprocess_hsv(x, img_size, bin) for x in train_data]
        # train_data_ = [preprocess_rgb(x, img_size, bin) for x in train_data]
        train_data_ = np.array(train_data_)

        param_grid = {
            'C': [0.01, 0.01, 0.1, 1, 10, 15, 20],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 
            'gamma' : ["scale", "auto"],
        }       

        grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": bin,
            "img_size": img_size,
            "model": grid_search           
        }
        )

print("DONE")
elapsed = time.time() - start
print(f"Eplased time: {elapsed}")

Processing: 100%|██████████| 6/6 [03:10<00:00, 31.73s/it]

DONE
Eplased time: 190.3766520023346


Chay them image size

In [ ]:
results = []
start = time.time()

ex_sizes = [512, 1024]
for bin in tqdm(bins, desc="Processing bin"):
    for img_size in ex_sizes:
        svm = SVC()

        # train_data_ = [preprocess_hsv(x, img_size, bin) for x in train_data]
        train_data_ = [preprocess_rgb(x, img_size, bin) for x in train_data]

        train_data_ = np.array(train_data_)

        param_grid = {
            'C': [0.01, 0.01, 0.1, 1, 10, 15, 20],
            'kernel': ['linear', 'rbf', 'poly', 'sigmoid'], 
            'gamma' : ["scale", "auto"],
        }       

        grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": bin,
            "img_size": img_size,
            "model": grid_search           
        }
        )

print("DONE")
elapsed = time.time() - start
print(f"Eplased time: {elapsed}")

In [4]:
best_model = None
best_score = -float('inf')  # Đặt giá trị thấp nhất ban đầu để so sánh

# Lặp qua tất cả các kết quả "trong results
for grid_search in results:
    if grid_search["model"].best_score_ > best_score:
        best_score = grid_search["model"].best_score_
        best_model = grid_search["model"].best_estimator_
        best_params = grid_search["model"].best_params_
        bin_ = grid_search["bin"]
        img_size_ = grid_search["img_size"]


# In kết quả
print("\nBest Overall Model:")
print("img_size:", img_size_)
print("bin:", bin_)
print("Best Parameters:", best_params)
print()
# save parameters
params = {
    "img_size": img_size_,
    "bin": bin_,
}
params.update(best_params)
print(params)

# #save model
# joblib.dump(best_model, './checkpoint/svm_best_model_rgb.pkl')
# #save params
# with open('./checkpoint/svm_params_rgb.json', 'w') as f:
#     json.dump(params, f)



Best Overall Model:
img_size: 512
bin: 128
Best Parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}

{'img_size': 512, 'bin': 128, 'C': 1, 'gamma': 'scale', 'kernel': 'rbf'}


# *KNN*

In [5]:
results = []
start = time.time()
for bin in tqdm(bins, desc="Processing"):
    for img_size in sizes:
        knn = KNeighborsClassifier()
        train_data_ = [preprocess_hsv(x, img_size, bin) for x in train_data]
        # train_data_ = [preprocess_rgb(x, img_size, bin) for x in train_data]
        train_data_ = np.array(train_data_)

        param_grid = {
            'n_neighbors': [1, 3, 5, 7, 9, 11, 13],         
            'weights': ['uniform', 'distance'],
            'metric': [
                'euclidean', 'manhattan', 'minkowski', 'chebyshev', 'cosine', 'hamming'
            ]
        }

        grid_search = GridSearchCV(estimator=knn, param_grid=param_grid, cv=5, scoring='accuracy')

        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": bin,
            "img_size": img_size,
            "model": grid_search           
        }
        )

print("DONE")
elapsed = time.time() - start
print(f"Eplased time: {elapsed}")

Processing: 100%|██████████| 6/6 [03:46<00:00, 37.70s/it]

DONE
Eplased time: 226.18544507026672


In [6]:
best_model = None
best_score = -float('inf')

for grid_search in results:
    if grid_search["model"].best_score_ > best_score:
        best_score = grid_search["model"].best_score_
        best_model = grid_search["model"].best_estimator_
        best_params = grid_search["model"].best_params_
        bin_ = grid_search["bin"]
        img_size_ = grid_search["img_size"]


print("\nBest Overall Model:")
print("img_size:", img_size_)
print("bin:", bin_)
print("Best Parameters:", best_params)
print()

params = {
    "img_size": img_size_,
    "bin": bin_,
}
params.update(best_params)
print(params)

#save params
with open('./checkpoint/knn_params2.json', 'w') as f:
    json.dump(params, f)

# save model 
joblib.dump(best_model, './checkpoint/knn_best_model2.pkl')


Best Overall Model:
img_size: 64
bin: 64
Best Parameters: {'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}

{'img_size': 64, 'bin': 64, 'metric': 'euclidean', 'n_neighbors': 9, 'weights': 'uniform'}


# *Random Forest*

In [12]:
results = []

start_time = time.time()

for bin in tqdm(bins, desc="Bins"):
    for img_size in tqdm(sizes, desc="Image Sizes", leave=False):
        rf = RandomForestClassifier(random_state=42, n_jobs=-1)  

        # train_data_ = [preprocess_hsv(x, img_size, bin) for x in train_data]
        train_data_ = [preprocess_rgb(x, img_size, bin) for x in train_data]
        train_data_ = np.array(train_data_)

        param_grid = {
            'n_estimators': [50, 100, 150, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10, 15, 20],
            'min_samples_leaf': [1, 2, 4, 6, 8, 10],
        }

        grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

        grid_search.fit(train_data_, train_labels)
        results.append({
            "bin": bin,
            "img_size": img_size,
            "model": grid_search           
        })

end_time = time.time()
elapsed_time = end_time - start_time

print("DONE")
print(f"Total Time Taken: {elapsed_time:.2f} seconds")

Bins: 100%|██████████| 6/6 [09:03<00:00, 90.64s/it] 

DONE
Total Time Taken: 543.82 seconds


In [15]:
best_model = None
best_score = -float('inf')

for grid_search in results:
    if grid_search["model"].best_score_ > best_score:
        best_score = grid_search["model"].best_score_
        best_model = grid_search["model"].best_estimator_
        best_params = grid_search["model"].best_params_
        bin_ = grid_search["bin"]
        img_size_ = grid_search["img_size"]


print("\nBest Overall Model:")
print("img_size:", img_size_)
print("bin:", bin_)
print("Best Parameters:", best_params)

params = {
    "img_size": img_size_,
    "bin": bin_,
}
params.update(best_params)
print(params)

#save params
# with open('./checkpoint/rf_params2.json', 'w') as f:
#     json.dump(params, f)

# # save model 
# joblib.dump(best_model, './checkpoint/rf_best_model2.pkl')

with open('./checkpoint/rf_params_rgb.json', 'w') as f:
    json.dump(params, f)

# save model 
joblib.dump(best_model, './checkpoint/rf_best_model_rgb.pkl')


Best Overall Model:
img_size: 256
bin: 8
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
{'img_size': 256, 'bin': 8, 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}


['./checkpoint/rf_best_model_rgb.pkl']